In [1]:
import pandas as pd
# read data
Workers = pd.read_csv("../Workers.csv")
Skills = pd.read_csv("../WSkills.csv")
# Job Recommended for
Job = {'Worker':'Data Scientist'}
JobSkills = [{'Skill':'python','Skill_Rating':3},{'Skill':'SQL','Skill_Rating':5},{'Skill':'Cloud','Skill_Rating':2},{'Skill':'HTML','Skill_Rating':3}]

JobSkillIds = [skill['Skill'] for skill in JobSkills] # get skills id 
JobSkillRatingSum = sum([skill['Skill_Rating'] for skill in JobSkills]) # sum Skill Rating
# filter and merge Worker data
filtered_skills = Skills[Skills['Skill'].isin(JobSkillIds)]# Filter Skills to include skill in JobSkills list
worker_skill_ratings = filtered_skills.groupby('Worker')['Skill_Rating'].sum().reset_index()# Group the data by worker and calculate the sum
worker_skill_ratings['Skill_Rating'] = worker_skill_ratings['Skill_Rating'].div(JobSkillRatingSum).mul(100)
Workers['Profession'] = Workers.apply(lambda x: 1 if x['Profession'].lower().find(Job['Worker'].lower())!=-1 else 0, axis=1)

Workers_Data = pd.merge(Workers, worker_skill_ratings, left_on='id', right_on='Worker')
Workers_Data = Workers_Data.rename(columns={'Skill_Rating': 'Skill_Percentage_Required'})
Workers_Data.drop(['Worker','Rating','Nbr_Rating'], axis=1, inplace=True)
# calculate Percentage
Workers_Data['Percentage'] = Workers_Data.apply(lambda x: x['Skill_Percentage_Required']+x['Profession']*5, axis=1)
Workers_Data['Recommended'] = Workers_Data.apply(lambda x:1 if x['Percentage']>=30 else 0, axis=1)
Workers_Data

,id,Profession,Skill_Percentage_Required,Percentage,Recommended
0,1,1,15.384615,20.384615,0
1,3,1,23.076923,28.076923,0
2,4,1,38.461538,43.461538,1
3,9,1,30.769231,35.769231,1
4,10,1,23.076923,28.076923,0
...,...,...,...,...,...
3095,11993,0,23.076923,23.076923,0
3096,11995,0,30.769231,30.769231,1
3097,11996,0,15.384615,15.384615,0
3098,11998,0,30.769231,30.769231,1


In [2]:
 # Train the model
 # Init
Workers_Data.drop(['Percentage'],axis='columns',inplace=True)
target = Workers_Data.Recommended # prcise the target
inputs = Workers_Data.drop('Recommended',axis='columns') # Init the triening input


In [3]:
 # split dataset to training and testing
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(inputs,target,test_size=0.2)
 # model creat
from sklearn.naive_bayes import GaussianNB
model = GaussianNB()

In [4]:
 # Train
model.fit(X_train,y_train) # train the model

GaussianNB()

In [5]:
model.score(X_test,y_test) # test the model

1.0

In [6]:
model.predict_proba(inputs) # show proba of predections

array([[0.98675058, 0.01324942],
       [0.78208   , 0.21792   ],
       [0.00189186, 0.99810814],
       ...,
       [0.98755801, 0.01244199],
       [0.10112448, 0.89887552],
       [0.79273234, 0.20726766]])

In [7]:
output = model.predict(inputs) # predict if personn servaved or not

In [8]:
concatenated = pd.concat([inputs, pd.DataFrame(output, columns=["predicted_output"])], axis=1)
concatenated

,id,Profession,Skill_Percentage_Required,predicted_output
0,1,1,15.384615,0
1,3,1,23.076923,0
2,4,1,38.461538,1
3,9,1,30.769231,1
4,10,1,23.076923,0
...,...,...,...,...
3095,11993,0,23.076923,0
3096,11995,0,30.769231,1
3097,11996,0,15.384615,0
3098,11998,0,30.769231,1


In [9]:
 #Save the model
import pickle
pickle.dump(model, open('../model_Recommend_Worker.pkl', 'wb'))